In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate


function = {
    "name": "create_quiz",
    "description": "function that takes a list of questions and answers and returns a quiz",
    "parameters": {
        "type": "object",
        "properties": {
            "questions": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                        },
                        "answers": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "answer": {
                                        "type": "string",
                                    },
                                    "correct": {
                                        "type": "boolean",
                                    },
                                },
                                "required": ["answer", "correct"],
                            },
                        },
                    },
                    "required": ["question", "answers"],
                },
            }
        },
        "required": ["questions"],
    },
}


llm = ChatOpenAI(
    temperature=0.1,
).bind(
    function_call={
        "name": "create_quiz",
    },
    functions=[
        function,
    ],
)

prompt = PromptTemplate.from_template("Make a quiz about {city}")

chain = prompt | llm

response = chain.invoke({"city": "Seoul"})


response = response.additional_kwargs["function_call"]["arguments"]

response

'{\n  "questions": [\n    {\n      "question": "What is the capital city of South Korea?",\n      "answers": [\n        {\n          "answer": "Seoul",\n          "correct": true\n        },\n        {\n          "answer": "Busan",\n          "correct": false\n        },\n        {\n          "answer": "Incheon",\n          "correct": false\n        },\n        {\n          "answer": "Daegu",\n          "correct": false\n        }\n      ]\n    },\n    {\n      "question": "Which river flows through Seoul?",\n      "answers": [\n        {\n          "answer": "Han River",\n          "correct": true\n        },\n        {\n          "answer": "Nakdong River",\n          "correct": false\n        },\n        {\n          "answer": "Geum River",\n          "correct": false\n        },\n        {\n          "answer": "Yeongsan River",\n          "correct": false\n        }\n      ]\n    },\n    {\n      "question": "What is the tallest building in Seoul?",\n      "answers": [\n        {\n 

In [5]:
import json

for question in json.loads(response)["questions"]:
    print(question)

{'question': 'What is the capital city of South Korea?', 'answers': [{'answer': 'Seoul', 'correct': True}, {'answer': 'Busan', 'correct': False}, {'answer': 'Incheon', 'correct': False}, {'answer': 'Daegu', 'correct': False}]}
{'question': 'Which river flows through Seoul?', 'answers': [{'answer': 'Han River', 'correct': True}, {'answer': 'Nakdong River', 'correct': False}, {'answer': 'Geum River', 'correct': False}, {'answer': 'Yeongsan River', 'correct': False}]}
{'question': 'What is the tallest building in Seoul?', 'answers': [{'answer': 'Lotte World Tower', 'correct': True}, {'answer': 'N Seoul Tower', 'correct': False}, {'answer': '63 Building', 'correct': False}, {'answer': 'Seoul Sky', 'correct': False}]}
{'question': 'Which palace is located in the heart of Seoul?', 'answers': [{'answer': 'Gyeongbokgung Palace', 'correct': True}, {'answer': 'Changdeokgung Palace', 'correct': False}, {'answer': 'Deoksugung Palace', 'correct': False}, {'answer': 'Changgyeonggung Palace', 'correc